<a href="https://colab.research.google.com/github/yingzibu/drug_design_JAK/blob/main/VAE/vae_gentrl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SMILES -> SELFIES -> ENCODER -> latent z -> SELFIES

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y

# !pip install torch-geometric==1.7.1
# !pip install git+https://github.com/pyg-team/pytorch_geometric.git

!pip install pubchempy --quiet
!pip install transformers --quiet
!pip install cairosvg --quiet
!pip install varname --quiet
!pip install Cython --quiet
!pip install rdkit --quiet
!pip install molsets --quiet
!pip install pathlib --quiet
!pip install xgboost==1.6.1 --quiet
!pip install dgllife --quiet

# cpu version, usable for calculation
# !pip install dgl==1.1 --quiet

!pip install molvs --quiet


In [ ]:
# !pip install git+https://github.com/pyg-team/pytorch_geometric.git --quiet

!pip install git+https://github.com/pyg-team/pytorch_geometric.git --quiet
!pip install molsets --quiet # gentrl essential

!pip uninstall dgl -y
!pip install  dgl -f https://data.dgl.ai/wheels/cu118/repo.html --quiet
!pip install  dglgo -f https://data.dgl.ai/wheels-test/repo.html --quiet

!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html --quiet
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html --quiet
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html --quiet

In [8]:
ls

drive/  sample_data/


In [9]:
!python --version
import torch
print('torch version: ', torch.__version__)
print('cuda available: ', torch.cuda.is_available())
import dgl
print('dgl version: ', dgl.__version__)
import dgllife
print('dgllife version: ', dgllife.__version__)
import rdkit
print('rdkit version: ', rdkit.__version__)
import molvs
print('molvs version: ', molvs.__version__)
import matplotlib
print('matplotlib version: ', matplotlib.__version__)

# def warn(*args, **kwargs):
#     pass
# import warnings
# warnings.warn = warn

# Python 3.10.6
# torch version:  2.0.1+cu118
# cuda available:  True
# Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
# DGL backend not selected or invalid.  Assuming PyTorch for now.
# dgl version:  1.1.1+cu118
# dgllife version:  0.3.2
# rdkit version:  2022.09.5
# molvs version:  0.1.1
# matplotlib version:  3.7.1

Python 3.10.12
torch version:  2.0.1+cu118
cuda available:  True
dgl version:  1.1.0
dgllife version:  0.3.2
rdkit version:  2023.03.2
molvs version:  0.1.1
matplotlib version:  3.7.1


In [10]:
model_path =  '/content/drive/MyDrive/JAK_ML/model/'
# model_path = 'model/'


import pickle
from rdkit.Chem import MACCSkeys
from rdkit import Chem
import pandas as pd
def smile_list_to_MACCS(smi_list):
    MACCS_list = []
    # maccs = None
    if isinstance(smi_list, str):
        smi_list = [smi_list]
    for smi in smi_list:
        mol = Chem.MolFromSmiles(smi)
        maccs = list(MACCSkeys.GenMACCSKeys(mol).ToBitString())
        MACCS_list.append(maccs)

    header = ['bit' + str(i) for i in range(167)]
    df = pd.DataFrame(MACCS_list,columns=header)
        # maccs = df.values
    return df.values
def simp_model_predict(smi_list, enzyme, ml, model_path=model_path):
    # Works for SVM_poly, RF, XGBoost
    if isinstance(smi_list, str):
        smi_list = [smi_list]
    modelname = ml + '_' + enzyme + '.sav'
    model = pickle.load(open(model_path+modelname, 'rb'))
    # maccs = smile_list_to_MACCS(smi)
    pred = model.predict(smile_list_to_MACCS(smi_list))
    prob = model.predict_proba(smile_list_to_MACCS(smi_list))
#     type(prob[0][1])
#     print(ml, pred, prob, prob[0][1])
    return prob[:, 1], pred
import os
def create_path(path):
    # Check whether the specified path exists or not
    isExist = os.path.exists(path)
    #printing if the path exists or not
    print(path, ' folder is in directory: ', isExist)
    if not isExist:
    # Create a new directory because it does not exist
        os.makedirs(path)
        print(path, " is created!")

CoGT prediction, threshold 10 uM

In [14]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

smiles = {}
# smiles['test'] = max_len_smi
# smiles['remove_k'] = new
smiles['MMT3-72'] = 'O=C(NCCC(O)=O)C(C=C1)=CC=C1/N=N/C(C=C2C(O)=O)=CC=C2OCCOC3=CC=C(NC4=NC=C(C)C(NC5=CC=CC(S(NC(C)(C)C)(=O)=O)=C5)=N4)C=C3'
smiles['M1'] = 'OCCOC1=CC=C(NC2=NC=C(C)C(NC3=CC=CC(S(N)(=O)=O)=C3)=N2)C=C1'
smiles['M2'] = 'OCCOC1=CC=C(NC2=NC=C(C)C(NC3=CC=CC(S(NC(C)(C)C)(=O)=O)=C3)=N2)C=C1'
smiles['M3'] = 'OC(COC1=CC=C(NC2=NC=C(C)C(NC3=CC=CC(S(NC(C)(C)C)(=O)=O)=C3)=N2)C=C1)=O'
smiles['M4'] = 'O=C(NCCC(O)=O)C(C=C1)=CC=C1/N=N/C(C=C2C(O)=O)=CC=C2OCCOC3=CC=C(NC4=NC=C(C)C(NC5=CC=CC(S(N)(=O)=O)=C5)=N4)C=C3'
smiles['M5'] = 'OC1=CC=C(NC2=NC=C(C)C(NC3=CC=CC(S(NC(C)(C)C)(=O)=O)=C3)=N2)C=C1'

probs, preds = simp_model_predict(list(smiles.values()), 'JAK2', 'RF')


In [15]:
preds

array([0, 0, 0, 1, 0, 0])

MTATFP prediction, if convert_binary=True, you could define the thred_CoGT, default = 250nM

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from dgllife.model import model_zoo
from dgllife.utils import smiles_to_bigraph
from dgllife.utils import AttentiveFPAtomFeaturizer
from dgllife.utils import AttentiveFPBondFeaturizer
from dgllife.data import MoleculeCSVDataset
import dgl
import matplotlib
import matplotlib.cm as cm
from IPython.display import SVG, display
from rdkit import Chem
from rdkit.Chem import rdDepictor
from rdkit.Chem.Draw import rdMolDraw2D
from datetime import datetime

if torch.cuda.is_available():
    print('use GPU')
    device = 'cuda'
else:
    print('use CPU')
    device = 'cpu'

def collate_molgraphs(data):
    assert len(data[0]) in [3, 4], \
        'Expect the tuple to be of length 3 or 4, got {:d}'.format(len(data[0]))
    if len(data[0]) == 3:
        smiles, graphs, labels = map(list, zip(*data))
        masks = None
    else:
        smiles, graphs, labels, masks = map(list, zip(*data))

    bg = dgl.batch(graphs)
    bg.set_n_initializer(dgl.init.zero_initializer)
    bg.set_e_initializer(dgl.init.zero_initializer)
    labels = torch.stack(labels, dim=0)

    if masks is None:
        masks = torch.ones(labels.shape)
    else:
        masks = torch.stack(masks, dim=0)
    return smiles, bg, labels, masks


def handle_less_symbol(a_string): # '<50'
#     l = a_string.split()
    s = a_string[1:]
    try:
        s = int(s)
        return s
    except:
        print('cannot convert s')
        return s

def threshold_IC50(row, enzyme, thres=250):
    num = row[enzyme]
    if isinstance(num, str):
        try:
            num = float(num)
        except: num = handle_less_symbol(num)
    try:
        if num > thres: return 0 # noninhibitor
        else: return 1
    except: # <500 type, cannot convert
        print('error', num)

# !cd /Users/uranaieiko_1/Desktop/JAK_design/MTATFP/Data



fn = '/content/drive/MyDrive/JAK-MTATFP/MTATFP/Model/MTATFP_jak.pt'

def MTATFP_predict(df_smiles, convert_binary=False, thres_CoGT=250, fn=fn): # modelfile is fn
    now = datetime.now() # current date and time
    date_time = now.strftime("%m_%d_%Y__%H:%M:%S")
    bin_path = 'bins/'
    create_path(bin_path)
    cache_file_path = bin_path + date_time + '.bin'

    atom_featurizer = AttentiveFPAtomFeaturizer(atom_data_field='hv')
    bond_featurizer = AttentiveFPBondFeaturizer(bond_data_field='he')
    n_feats = atom_featurizer.feat_size('hv')
    e_feats = bond_featurizer.feat_size('he')

    data_df = pd.DataFrame()
    data_df['SMILES'] = df_smiles

    for task_name in ['pIC50_JAK1','pIC50_JAK2','pIC50_JAK3','pIC50_TYK2']:
        data_df[task_name] = 888888888888888 # Random num

    extest_datasets = MoleculeCSVDataset(data_df,
                            smiles_to_graph=smiles_to_bigraph,
                            node_featurizer=atom_featurizer,
                            edge_featurizer= bond_featurizer,
                            smiles_column='SMILES',
                            cache_file_path=cache_file_path,
                            task_names=['pIC50_JAK1','pIC50_JAK2',
                                        'pIC50_JAK3','pIC50_TYK2'],
                            load=True,init_mask=True
                            )

    # print('ExTest sets: ',len(extest_datasets))

    # fn = model_file
    model = model_zoo.AttentiveFPPredictor(node_feat_size=n_feats,
                                    edge_feat_size=e_feats,
                                    num_layers=2,
                                    num_timesteps=1,
                                    graph_feat_size=300,
                                    n_tasks=4,
                                        )
    model.load_state_dict(torch.load(fn,map_location=torch.device(device)))

    for i in range(len(data_df)):
        smiles, g, label, _ = extest_datasets[i]
        g = dgl.batch([g])
        atom_feats, bond_feats = g.ndata.pop('hv'), g.edata.pop('he')
        preds, _ = model(g, atom_feats, bond_feats, get_node_weight=True)
        jak_list = [round(10**(-1.0 * i), 4) for i in preds[0].tolist()]
        for enzyme, ic50 in zip(['JAK1', 'JAK2', 'JAK3', 'TYK2'], jak_list):
            data_df.loc[i, enzyme] = ic50
    # print(data_df[['SMILES', 'JAK1', 'JAK2', 'JAK3', 'TYK2']])
    data_df = data_df[['SMILES', 'JAK1', 'JAK2', 'JAK3', 'TYK2']]
    !rm -rf bins/
    if convert_binary == True:
        print(f'BINARY CLASSIFICATION: threshold is set {thres_CoGT} nM')
        new_df = pd.DataFrame()
        for enzyme in ['JAK1', 'JAK2', 'JAK3', 'TYK2']:
            new_df['SMILES'] = data_df['SMILES']
            new_df[enzyme] = data_df[enzyme]
            new_df[enzyme] = data_df.apply(
                    lambda row: threshold_IC50(row, enzyme, thres_CoGT), axis=1)

        return new_df
        # print(f'Data processed for CoGT binary ({thres_CoGT}nM) saved in ',
        #           save_path+f'for_CoGT_{thres_CoGT}nM_'+savename)
        # new_df.to_csv(save_path+f'for_CoGT_{thres_CoGT}nM_'+savename,
        #               index=False)

    return data_df

use GPU


In [19]:
MTATFP_predict(list(smiles.values()), convert_binary=True)

bins/  folder is in directory:  False
bins/  is created!
Processing dgl graphs from scratch...
BINARY CLASSIFICATION: threshold is set 250 nM


,SMILES,JAK1,JAK2,JAK3,TYK2
0,O=C(NCCC(O)=O)C(C=C1)=CC=C1/N=N/C(C=C2C(O)=O)=...,0,0,1,0
1,OCCOC1=CC=C(NC2=NC=C(C)C(NC3=CC=CC(S(N)(=O)=O)...,1,0,0,0
2,OCCOC1=CC=C(NC2=NC=C(C)C(NC3=CC=CC(S(NC(C)(C)C...,1,1,0,0
3,OC(COC1=CC=C(NC2=NC=C(C)C(NC3=CC=CC(S(NC(C)(C)...,0,1,0,0
4,O=C(NCCC(O)=O)C(C=C1)=CC=C1/N=N/C(C=C2C(O)=O)=...,1,0,0,0
5,OC1=CC=C(NC2=NC=C(C)C(NC3=CC=CC(S(NC(C)(C)C)(=...,1,1,0,0


In [20]:
cd /content/drive/MyDrive/JAK_ML/

/content/drive/MyDrive/JAK_ML


In [21]:
# from torch_geometric.data import DataLoader

In [22]:
from gentrl_prerun import *

/usr/local/lib/python3.10/dist-packages/moses/metrics/utils.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  _mcf.append(_pains, sort=True)['smarts'].values]


device:  cuda
If cuda available: True
functions loaded
data.shape: (27680, 2)


In [30]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


In [34]:
create_path('A_DATA/')

A_DATA/  folder is in directory:  False
A_DATA/  is created!


In [35]:
ls

 A_DATA/
 A_JAK_design/
 atari_deep_Q_learning/
'Build an Autoencoder.ipynb'
 chemberta/
 chemberta_train_cuda.ipynb
 chemical_vae/
'chemical VAE smi to continuous decode to smi.ipynb'
'Colab Notebooks'/
 Colab_Notebooks/
 compound_new.xlsx
 conditionalVAE.ipynb
 conplex/
 cs285_f2022/
 Data/
 dataset.png
 denoising_autoencoder.png
 DL_Pytorch_Book/
 DL_Pytorch_Chapter8.ipynb
 Drug_design/
 drugGPT/
 drugGPT.ipynb
 drug_smile.csv.gsheet
 EquiBind-main/
 function.cpython-310.pyc
 gentrl.ipynb
 graph.pptx
 GraphVAE.ipynb
 helper.py
 JAK_ML/
 JAK-MTATFP/
 mnist_dataset.csv
 models/
'Nick & Shalaka Sun Lab Capstone'@
 previous_files/
 py3Dmol.ipynb
 result.tsv
 smile_new.xlsx
 test_config.yaml
 test_new.tsv
 vina/


In [31]:
data = pd.read_csv('/content/drive/MyDrive/A_JAK_design/Data/chembl.csv')
data

,SMILES,QED
0,Cc1cc(-c2csc(N=C(N)N)n2)cn1C,0.608781
1,CC[C@H](C)[C@H](NC(=O)[C@H](CC(C)C)NC(=O)[C@@H...,0.016357
2,CCCC[C@@H]1NC(=O)[C@@H](NC(=O)[C@H](CC(C)C)NC(...,0.010798
3,CC(C)C[C@@H]1NC(=O)CNC(=O)[C@H](c2ccc(O)cc2)NC...,0.016726
4,Brc1cccc(Nc2ncnc3ccncc23)c1NCCN1CCOCC1,0.623114
...,...,...
2136181,CCCCCCCCCCCCCCCCCCPCCCCCCCCCCCCCC.F[PH](F)(F)(...,0.042174
2136182,C[n+]1cn([C@@H]2O[C@H](CO[P@@](=O)(S)OP(=O)([O...,0.103859
2136183,c1ccc(C2CC(C3CC(c4ccccc4)OC(c4ccccc4)C3)CC(c3c...,0.287634
2136184,CSCC[C@H](NC=O)C(=O)N[C@@H](CCCNC(=N)NS(=O)(=O...,0.006376


In [33]:
enc = gentrl.RNNEncoder(latent_size=50)
dec = gentrl.DilConvDecoder(latent_input_size=50)
model = gentrl.GENTRL(enc, dec, 50 * [('c', 20)], [('c', 20)], beta=0.001)
model.cuda();
md = gentrl.MolecularDataset(sources=[
    {'path':'/content/drive/MyDrive/A_JAK_design/Data/chembl.csv',
     'smiles': 'SMILES',
     'prob': 1,
     'QED' : 'QED',
    }],
    props=['QED'])

from torch.utils.data import DataLoader
train_loader = DataLoader(md, batch_size=50, shuffle=True, num_workers=1, drop_last=True)
model.train_as_vaelp(train_loader, lr=1e-4)

Just run cannot work, filter the data

In [36]:
import re
_atoms = ['He', 'Li', 'Be', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'Cl', 'Ar',
          'Ca', 'Ti', 'Cr', 'Fe', 'Ni', 'Cu', 'Ga', 'Ge', 'As', 'Se',
          'Br', 'Kr', 'Rb', 'Sr', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh',
          'Pd', 'Ag', 'Cd', 'Sb', 'Te', 'Xe', 'Ba', 'La', 'Ce', 'Pr',
          'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Er', 'Tm', 'Yb',
          'Lu', 'Hf', 'Ta', 'Re', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb',
          'Bi', 'At', 'Fr', 'Ra', 'Ac', 'Th', 'Pa', 'Pu', 'Am', 'Cm',
          'Bk', 'Cf', 'Es', 'Fm', 'Md', 'Lr', 'Rf', 'Db', 'Sg', 'Mt',
          'Ds', 'Rg', 'Fl', 'Mc', 'Lv', 'Ts', 'Og']


def get_tokenizer_re(atoms):
    return re.compile('('+'|'.join(atoms)+r'|\%\d\d|.)')

def smiles_tokenizer(line, atoms=None):
    """
    Tokenizes SMILES string atom-wise using regular expressions. While this
    method is fast, it may lead to some mistakes: Sn may be considered as Tin
    or as Sulfur with Nitrogen in aromatic cycle. Because of this, you should
    specify a set of two-letter atoms explicitly.

    Parameters:
         atoms: set of two-letter atoms for tokenization
    """
    if atoms is not None:
        reg = get_tokenizer_re(atoms)
    else:
        reg = _atoms_re
    return reg.split(line)[1::2]

__i2t = {
    0: 'unused', 1: '>', 2: '<', 3: '2', 4: 'F', 5: 'Cl', 6: 'N',
    7: '[', 8: '6', 9: 'O', 10: 'c', 11: ']', 12: '#',
    13: '=', 14: '3', 15: ')', 16: '4', 17: '-', 18: 'n',
    19: 'o', 20: '5', 21: 'H', 22: '(', 23: 'C',
    24: '1', 25: 'S', 26: 's', 27: 'Br' , 28: '@', 29: '/', 30: '.',
    31: 'P', 32: '+', 33: 'I', 34: 'Si', 35: 'B', 36: '\\',
    37: '7', 38: '8', 39: '9'
}
# 28: '@', 29: '/', 30: '.'

__t2i = {
    '>': 1, '<': 2, '2': 3, 'F': 4, 'Cl': 5, 'N': 6, '[': 7, '6': 8,
    'O': 9, 'c': 10, ']': 11, '#': 12, '=': 13, '3': 14, ')': 15,
    '4': 16, '-': 17, 'n': 18, 'o': 19, '5': 20, 'H': 21, '(': 22,
    'C': 23, '1': 24, 'S': 25, 's': 26, 'Br': 27 , '@': 28, '/': 29, '.': 30,
    'P': 31, '+': 32, 'I': 33, 'Si': 34, 'B': 35, '\\': 36,
    '7': 37, '8': 38, '9': 39
}
#, '@': 28, '/': 29, '.': 30

_atoms_re = get_tokenizer_re(_atoms)
# _atoms_re
def remove_ions(smi_string):
    ion_list = ['.[K+]', '[Li+].', '.[Na+]']
    ions_exist = False
    current_smi = smi_string
    for ion in ion_list:
        if ion in current_smi:
            ions_exist = True
            print('delete ion :', ion)
            current_smi = current_smi.replace(ion, '')
            try:
                mol = get_mol(current_smi)
            except:
                print('cannot delete since mol becomes invalid')
    if ions_exist == False: # No ions exist, did not modify
        return smi_string, smi_string
    else:

        return current_smi, smi_string

In [37]:
data

,SMILES,QED
0,Cc1cc(-c2csc(N=C(N)N)n2)cn1C,0.608781
1,CC[C@H](C)[C@H](NC(=O)[C@H](CC(C)C)NC(=O)[C@@H...,0.016357
2,CCCC[C@@H]1NC(=O)[C@@H](NC(=O)[C@H](CC(C)C)NC(...,0.010798
3,CC(C)C[C@@H]1NC(=O)CNC(=O)[C@H](c2ccc(O)cc2)NC...,0.016726
4,Brc1cccc(Nc2ncnc3ccncc23)c1NCCN1CCOCC1,0.623114
...,...,...
2136181,CCCCCCCCCCCCCCCCCCPCCCCCCCCCCCCCC.F[PH](F)(F)(...,0.042174
2136182,C[n+]1cn([C@@H]2O[C@H](CO[P@@](=O)(S)OP(=O)([O...,0.103859
2136183,c1ccc(C2CC(C3CC(c4ccccc4)OC(c4ccccc4)C3)CC(c3c...,0.287634
2136184,CSCC[C@H](NC=O)C(=O)N[C@@H](CCCNC(=N)NS(=O)(=O...,0.006376


In [ ]:
import statistics
smiles = data['SMILES']
to_max_num = 0
split_num  = 0
res = []
lens = []
pad_size = 50
unpassed_tokens = []
unpassed_smiles = []
filtered_smiles = []
for i in smiles:
    # print(type(i))
    smi_tokens = list(i)
    # print(len(smi_tokens))

    # print(len(smiles_tokenizer(i)))
    # CHECK THE LENGTH OF TOKEN, SEEMS MAX 438, using 50 may not be good
    if len(smi_tokens) > split_num:
        split_num = len(smi_tokens)
    if len(smiles_tokenizer(i)) > to_max_num:
        to_max_num = len(smiles_tokenizer(i))
        max_len_smi = i
    try:
        tokens = ([1] + [__t2i[tok]
                for tok in smiles_tokenizer(i, atoms=['Cl', 'Br', 'Si'])])
        filtered_smiles.append(i)
    except:
        i, _ = remove_ions(i) # remove ions
        try: # Try again after remove ions
            tokens = ([1] + [__t2i[tok]
                for tok in smiles_tokenizer(i, atoms=['Cl', 'Br', 'Si'])])
            filtered_smiles.append(i)
        except:
            unpassed_smiles.append(i)
            for tok in smiles_tokenizer(i, atoms=['Cl', 'Br', 'Si']):
                if tok not in __t2i.keys() and tok not in unpassed_tokens:
                    unpassed_tokens.append(tok)
                    print('unpassed token:', tok)
                    print(i)


    # print(tokens)
    lens.append(len(tokens))
    tokens += (pad_size - len(tokens)) * [2]
    res.append(tokens)
# print(lens)
# print(to_max_num, split_num)
print('max len token smi: ', max_len_smi)
print('mean length of token list: ', statistics.median(lens))
print(unpassed_smiles)

In [ ]:
len(filtered_smiles)
unpassed_tokens

In [80]:
filtered_smiles[0] in data['SMILES'].tolist()
filtered_dict = {}
for i in tqdm(filtered_smiles):
    # if i in data['SMILES'].tolist():
        # q = data['QED'][data['SMILES']== i].tolist()[0]
    filtered_dict[i] = qed(i)

# for i, j in filtered_dict.items():
#     print(i, j)

100%|██████████| 2131413/2131413 [1:21:28<00:00, 436.04it/s]


In [71]:
pd.DataFrame(list(filtered_dict.values()))

,0
0,0.608781
1,0.016357
2,0.010798
3,0.016726
4,0.623114
5,0.553150
6,0.794875
7,0.202084
8,0.149872
9,0.041432


In [72]:
new_df = pd.DataFrame()
new_df['SMILES'] = pd.DataFrame(list(filtered_dict.keys()))
new_df['QED'] = pd.DataFrame(list(filtered_dict.values()))

In [74]:
create_path('A_DATA/')
new_df

A_DATA/  folder is in directory:  True


,SMILES,QED
0,Cc1cc(-c2csc(N=C(N)N)n2)cn1C,0.608781
1,CC[C@H](C)[C@H](NC(=O)[C@H](CC(C)C)NC(=O)[C@@H...,0.016357
2,CCCC[C@@H]1NC(=O)[C@@H](NC(=O)[C@H](CC(C)C)NC(...,0.010798
3,CC(C)C[C@@H]1NC(=O)CNC(=O)[C@H](c2ccc(O)cc2)NC...,0.016726
4,Brc1cccc(Nc2ncnc3ccncc23)c1NCCN1CCOCC1,0.623114
5,COc1c(O)cc(O)c(C(=N)Cc2ccc(O)cc2)c1O,0.553150
6,CCOC(=O)c1cc2cc(C(=O)O)ccc2[nH]1,0.794875
7,CC(=O)O[C@H]1/C(C)=C\[C@@H]2OC(=O)[C@]3(C)O[C@...,0.202084
8,C[C@H](NC(=O)OCc1ccccc1)C(=O)N[C@@H](C)C(=O)NN...,0.149872
9,CO[C@H]1C[C@H](COC[C@H]2[C@@H](OC)C[C@H](O[C@H...,0.041432


In [79]:
from moses.metrics import mol_passes_filters, QED, SA, logP
from moses.metrics.utils import get_n_rings, get_mol
from tqdm import tqdm

def get_num_rings_6(mol):
    r = mol.GetRingInfo()
    return len([x for x in r.AtomRings() if len(x) > 6])

def penalized_logP(mol_or_smiles, masked=False, default=-5):
    mol = get_mol(mol_or_smiles)
    if mol is None:
        return default
    reward = logP(mol) - SA(mol) - get_num_rings_6(mol)
    if masked and not mol_passes_filters(mol):
        return default
    return reward

def qed(mol_or_smiles, default=-5):
    mol = get_mol(mol_or_smiles)
    if mol is None:
        return default
    return QED(mol)

tqdm.pandas()
new_df['plogP'] = new_df['SMILES'].progress_apply(penalized_logP)
new_df['QED'] = new_df['SMILES'].progress_apply(qed)


100%|██████████| 10/10 [00:00<00:00, 169.16it/s]


In [75]:
new_df.to_csv(f'A_DATA/filtered_chembl_for_gentrl.csv', index=False)

TEST GENTRL AGAIN using A_DATA/filtered_chembl_for_gentrl.csv

In [76]:
data_path = 'A_DATA/filtered_chembl_for_gentrl.csv'
filtered_data = pd.read_csv(data_path)

enc = gentrl.RNNEncoder(latent_size=50)
dec = gentrl.DilConvDecoder(latent_input_size=50)
model = gentrl.GENTRL(enc, dec, 50 * [('c', 20)], [('c', 20)], beta=0.001)
model.cuda();
md = gentrl.MolecularDataset(sources=[
    {'path': data_path,
     'smiles': 'SMILES',
     'prob': 1,
     'QED' : 'QED',
    }],
    props=['QED'])

from torch.utils.data import DataLoader
train_loader = DataLoader(md, batch_size=50, shuffle=True, num_workers=1, drop_last=True)
model.train_as_vaelp(train_loader, lr=1e-4)

Epoch 0 :
Epoch 2 :
Epoch 4 :
Epoch 6 :
Epoch 8 :


In [77]:
l

,SMILES,QED
0,Cc1cc(-c2csc(N=C(N)N)n2)cn1C,0.608781
1,CC[C@H](C)[C@H](NC(=O)[C@H](CC(C)C)NC(=O)[C@@H...,0.016357
2,CCCC[C@@H]1NC(=O)[C@@H](NC(=O)[C@H](CC(C)C)NC(...,0.010798
3,CC(C)C[C@@H]1NC(=O)CNC(=O)[C@H](c2ccc(O)cc2)NC...,0.016726
4,Brc1cccc(Nc2ncnc3ccncc23)c1NCCN1CCOCC1,0.623114
5,COc1c(O)cc(O)c(C(=N)Cc2ccc(O)cc2)c1O,0.553150
6,CCOC(=O)c1cc2cc(C(=O)O)ccc2[nH]1,0.794875
7,CC(=O)O[C@H]1/C(C)=C\[C@@H]2OC(=O)[C@]3(C)O[C@...,0.202084
8,C[C@H](NC(=O)OCc1ccccc1)C(=O)N[C@@H](C)C(=O)NN...,0.149872
9,CO[C@H]1C[C@H](COC[C@H]2[C@@H](OC)C[C@H](O[C@H...,0.041432
